<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b796c6f4ae220f8e34e51beef1ee59639681f76cfe3a281e34f99bba9fd41024
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-30 11:17:53
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.37 C
-------------------
Today PnL: 24.42 K (0.18%)
Current PnL: -22.88 L (-15.39%)
CY Booked + Current PnL: -11.44 L (-7.69%)
-------------------
Total profit:  1.02 L
Total loss:  -23.90 L
-------------------
Total Booked + Current PnL: 15.54 L (12.77%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.37%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 88.74 L (64.91%)
Deployed:  1.22 C
Current:  1.37 C
CAGR/XIRR %: 7.01%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
50,NATIONALUM,244.55,-44.49,59.0,H-MC,12.17,111062.0,10558.0,18547.0,2.77,10.51,16.70,28.96,79.0,0.57,0.81,48.04,MH,ATH,METALS
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.41,188490.0,23820.0,19509.0,-0.56,14.47,10.35,26.31,89.0,1.22,1.38,23.60,X40N,NTT,AC
42,ITC,452.00,-38.77,44.0,H-LC,1.94,197189.0,-2949.0,23840.0,-0.98,-1.47,12.09,10.44,4.0,-0.12,1.44,4.01,X40,NTT,FMCG
84,VOLTAS,1530.00,-2.76,44.0,H-MC,2.86,204015.0,12273.0,25481.0,1.48,6.40,12.49,19.69,99.0,0.48,1.49,13.63,XY25,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182806.0,1663.0,43800.0,6.74,0.92,23.96,25.10,235.0,0.04,1.34,46.50,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-182.95,76.0,M-SC,7.15,95562.0,-6648.0,100034.0,3.24,-6.50,104.68,91.37,208.0,-0.07,0.70,55.13,XY25,NTT,FINANCE
79,UNIONBANK,163.00,-13.18,56.0,M-LC,7.10,156883.0,16043.0,28443.0,2.91,11.39,18.13,31.59,66.0,0.56,1.15,40.90,XY24,NTT,BANKS
11,BANKINDIA,190.00,-26.71,63.0,H-MC,8.88,187805.0,7997.0,104420.0,2.81,4.45,55.60,62.52,88.0,0.08,1.37,40.05,XR,NTT,BANKS
50,NATIONALUM,244.55,-44.49,59.0,H-MC,12.17,111062.0,10558.0,18547.0,2.77,10.51,16.70,28.96,79.0,0.57,0.81,48.04,MH,ATH,METALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GREENPANEL,537.0,191.99,39.0,M-SC,0.98,130418.0,-52660.0,131644.0,-3.17,-28.76,100.94,43.14,230.0,-0.40,0.95,22.68,XY24,NTT,MISC
85,WHIRLPOOL,2270.0,-46.98,25.0,M-SC,1.27,89542.0,-1956.0,80704.0,-3.14,-2.14,90.13,86.07,223.0,-0.02,0.66,30.03,XR,NTT,DURABLES
38,INDIGOPNTS,1408.0,100.33,28.0,M-SC,1.54,131192.0,-43367.0,43398.0,-2.99,-24.84,33.08,0.02,221.0,-1.00,0.96,14.11,OX40N,NTT,PAINTS
81,VAIBHAVGBL,521.0,47.11,43.0,H-SC,5.81,127543.0,-55232.0,166303.0,-2.73,-30.22,130.39,60.77,125.0,-0.33,0.93,14.24,XR,NTT,JEWELLERY
28,HATHWAY,31.4,867.39,27.0,H-SC,1.99,83160.0,-25560.0,105239.0,-2.33,-23.51,126.55,73.29,161.0,-0.24,0.61,11.24,XR,NTT,ENTERTAINMENT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182806.0,1663.0,43800.0,6.74,0.92,23.96,25.10,235.0,0.04,1.34,46.50,XY24,NTT,HEALTHCARE
37,INDIAMART,4850.92,-55.95,22.0,H-SC,9.96,123225.0,-111.0,133872.0,-1.77,-0.09,108.64,108.45,119.0,-0.00,0.90,22.24,AR,ATH,MISC
35,ICICIPRULI,790.00,-21.80,43.0,H-MC,2.21,134470.0,-1318.0,44066.0,1.62,-0.97,32.77,31.48,107.0,-0.03,0.98,11.03,X40,ATH,INSURANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,97.20,56.0,H-SC,10.81,124443.0,-18036.0,37557.0,0.02,-12.66,30.18,13.70,163.0,-0.48,0.91,46.88,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,51.0,H-SC,1.54,221202.0,-48465.0,84809.0,-0.32,-17.97,38.34,13.47,138.0,-0.57,1.62,11.88,XY24,NTT,PAINTS
18,CERA,9475.0,-22.20,38.0,H-SC,1.59,141657.0,-34246.0,76268.0,0.01,-19.47,53.84,23.89,149.0,-0.45,1.04,22.34,OX40N,NTT,CERAMICS
66,SIS,528.0,2082.90,40.0,H-SC,4.02,88090.0,-22942.0,46547.0,0.07,-20.66,52.84,21.26,156.0,-0.49,0.64,19.02,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182806.0,1663.0,43800.0,6.74,0.92,23.96,25.10,235.0,0.04,1.34,46.50,XY24,NTT,HEALTHCARE
60,SAIL,228.00,45.15,56.0,M-MC,11.16,228923.0,3961.0,163703.0,0.93,1.76,71.51,74.53,192.0,0.02,1.67,34.58,XY24,BTT,STEEL
32,HINDZINC,730.22,27.24,63.0,M-LC,3.38,207538.0,2462.0,110099.0,2.35,1.20,53.05,54.89,52.0,0.02,1.52,25.59,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.17,46.0,H-MC,3.41,188490.0,23820.0,19509.0,-0.56,14.47,10.35,26.31,89.0,1.22,1.38,23.60,X40N,NTT,AC
84,VOLTAS,1530.00,-2.76,44.0,H-MC,2.86,204015.0,12273.0,25481.0,1.48,6.40,12.49,19.69,99.0,0.48,1.49,13.63,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-44.49,59.0,H-MC,12.17,111062.0,10558.0,18547.0,2.77,10.51,16.70,28.96,79.0,0.57,0.81,48.04,MH,ATH,METALS
11,BANKINDIA,190.00,-26.71,63.0,H-MC,8.88,187805.0,7997.0,104420.0,2.81,4.45,55.60,62.52,88.0,0.08,1.37,40.05,XR,NTT,BANKS
32,HINDZINC,730.22,27.24,63.0,M-LC,3.38,207538.0,2462.0,110099.0,2.35,1.20,53.05,54.89,52.0,0.02,1.52,25.59,X5K,ATH,METALS
37,INDIAMART,4850.92,-55.95,22.0,H-SC,9.96,123225.0,-111.0,133872.0,-1.77,-0.09,108.64,108.45,119.0,-0.00,0.90,22.24,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.98,25.0,M-SC,1.27,89542.0,-1956.0,80704.0,-3.14,-2.14,90.13,86.07,223.0,-0.02,0.66,30.03,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,29.0,H-LC,6.15,206642.0,-45126.0,98341.0,0.24,-17.92,47.59,21.13,27.0,-0.46,1.51,11.15,X40,ATH,PAINTS
44,JIOFIN,387.00,-6.42,29.0,H-LC,13.03,254302.0,-13063.0,78528.0,0.51,-4.89,30.88,24.48,48.0,-0.17,1.86,47.40,XY24,BTT,FINANCE
74,TCS,4476.75,-31.58,30.0,H-LC,12.27,273277.0,-72679.0,147542.0,0.38,-21.01,53.99,21.64,1.0,-0.49,2.00,0.38,X40,ATH,IT
83,VBL,671.64,-21.10,33.0,H-LC,8.82,284101.0,-31741.0,144409.0,-1.42,-10.05,50.83,35.67,5.0,-0.22,2.08,2.41,X40N,ATH,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,3.97,204651.0,151.0,41258.0,0.39,0.07,20.16,20.25,10.0,0.00,1.50,8.81,X40N,BTT,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-6.94,43.0,H-LC,1.62,230828.0,-15323.0,94524.0,-0.55,-6.23,40.95,32.18,16.0,-0.16,1.69,27.57,X200,ATH,IT
42,ITC,452.0,-38.77,44.0,H-LC,1.94,197189.0,-2949.0,23840.0,-0.98,-1.47,12.09,10.44,4.0,-0.12,1.44,4.01,X40,NTT,FMCG
65,SIEMENS,4671.5,-2.61,44.0,H-LC,2.39,156385.0,-29710.0,77192.0,1.01,-15.96,49.36,25.51,15.0,-0.38,1.14,15.24,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.79,40.0,H-LC,3.76,213252.0,2886.0,25889.0,-0.42,1.37,12.14,13.68,37.0,0.11,1.56,18.12,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-23.21,36.0,H-LC,3.97,204651.0,151.0,41258.0,0.39,0.07,20.16,20.25,10.0,0.00,1.50,8.81,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.27,273277.0,-72679.0,147542.0,0.38,-21.01,53.99,21.64,1.0,-0.49,2.00,0.38,X40,ATH,IT
83,VBL,671.64,-21.10,33.0,H-LC,8.82,284101.0,-31741.0,144409.0,-1.42,-10.05,50.83,35.67,5.0,-0.22,2.08,2.41,X40N,ATH,FMCG
80,UNITDSPR,1644.00,-14.37,45.0,H-LC,7.20,224477.0,-9667.0,58297.0,-0.26,-4.13,25.97,20.77,86.0,-0.17,1.64,2.50,X40N,NTT,BREWERIES
42,ITC,452.00,-38.77,44.0,H-LC,1.94,197189.0,-2949.0,23840.0,-0.98,-1.47,12.09,10.44,4.0,-0.12,1.44,4.01,X40,NTT,FMCG
30,HCLTECH,1943.91,-3.66,38.0,H-LC,10.12,220145.0,-21779.0,85042.0,1.07,-9.00,38.63,26.15,8.0,-0.26,1.61,4.03,X40,ATH,IT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-2.61,44.0,H-LC,2.39,156385.0,-29710.0,77192.0,1.01,-15.96,49.36,25.51,15.0,-0.38,1.14,15.24,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,44.0,H-LC,1.94,197189.0,-2949.0,23840.0,-0.98,-1.47,12.09,10.44,4.0,-0.12,1.44,4.01,X40,NTT,FMCG
19,CIPLA,1795.00,-23.21,36.0,H-LC,3.97,204651.0,151.0,41258.0,0.39,0.07,20.16,20.25,10.0,0.00,1.50,8.81,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.16,29.0,H-LC,6.15,206642.0,-45126.0,98341.0,0.24,-17.92,47.59,21.13,27.0,-0.46,1.51,11.15,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.79,40.0,H-LC,3.76,213252.0,2886.0,25889.0,-0.42,1.37,12.14,13.68,37.0,0.11,1.56,18.12,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,30.0,H-LC,12.27,273277.0,-72679.0,147542.0,0.38,-21.01,53.99,21.64,1.0,-0.49,2.00,0.38,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.20,308850.0,-4038.0,175736.0,0.57,-1.29,56.90,54.87,3.0,-0.02,2.26,5.27,X40,BTT,IT
42,ITC,452.00,-38.77,44.0,H-LC,1.94,197189.0,-2949.0,23840.0,-0.98,-1.47,12.09,10.44,4.0,-0.12,1.44,4.01,X40,NTT,FMCG
83,VBL,671.64,-21.10,33.0,H-LC,8.82,284101.0,-31741.0,144409.0,-1.42,-10.05,50.83,35.67,5.0,-0.22,2.08,2.41,X40N,ATH,FMCG
1,ABB,7934.00,-39.96,48.0,H-LC,7.68,251040.0,-10579.0,129788.0,1.09,-4.04,51.70,45.57,7.0,-0.08,1.84,6.82,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,40.27,78720.0,-34829.0,74878.0,1.41,-30.67,95.12,35.27,268.0,-0.47,0.58,92.67,XR,NTT,HOTELS
49,MASFIN,398.61,-20.51,35.0,H-SC,12.29,90090.0,-7890.0,29495.0,-0.78,-8.05,32.74,22.05,152.0,-0.27,0.66,30.95,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-182.95,76.0,M-SC,7.15,95562.0,-6648.0,100034.0,3.24,-6.50,104.68,91.37,208.0,-0.07,0.70,55.13,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-44.49,59.0,H-MC,12.17,111062.0,10558.0,18547.0,2.77,10.51,16.70,28.96,79.0,0.57,0.81,48.04,MH,ATH,METALS
78,UJJIVANSFB,60.00,97.20,56.0,H-SC,10.81,124443.0,-18036.0,37557.0,0.02,-12.66,30.18,13.70,163.0,-0.48,0.91,46.88,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1244.50,69.0,M-SC,14.28,182806.0,1663.0,43800.0,6.74,0.92,23.96,25.10,235.0,0.04,1.34,46.50,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-182.95,76.0,M-SC,7.15,95562.0,-6648.0,100034.0,3.24,-6.50,104.68,91.37,208.0,-0.07,0.70,55.13,XY25,NTT,FINANCE
60,SAIL,228.00,45.15,56.0,M-MC,11.16,228923.0,3961.0,163703.0,0.93,1.76,71.51,74.53,192.0,0.02,1.67,34.58,XY24,BTT,STEEL
50,NATIONALUM,244.55,-44.49,59.0,H-MC,12.17,111062.0,10558.0,18547.0,2.77,10.51,16.70,28.96,79.0,0.57,0.81,48.04,MH,ATH,METALS
32,HINDZINC,730.22,27.24,63.0,M-LC,3.38,207538.0,2462.0,110099.0,2.35,1.20,53.05,54.89,52.0,0.02,1.52,25.59,X5K,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.23
1,25,43.71
2,50,75.10


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.77
LC    32.54
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.40
X40      14.79
X40N     11.38
XR       11.11
XY25     10.25
AR        8.19
OX40N     7.88
X200      1.69
MH        1.63
X5K       1.52
SR        1.20
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.22
H-LC    25.94
H-MC    20.78
M-SC    13.08
M-LC     5.54
M-MC     2.65
L-SC     1.47
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.05,-6.86,42.13
IT,12.74,-20.82,86.69
FINANCE,10.16,-18.28,69.88
MISC,6.92,-21.39,88.87
BANKS,6.31,-15.61,76.43
PAINTS,5.63,-20.07,37.97
ELECTRICAL,5.49,-9.70,48.19
INSURANCE,3.80,-6.47,46.27
AC,3.58,1.04,15.52


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3227297.0,22
XR,1393631.0,14
AR,1211668.0,9
X40,826307.0,10
X40N,651101.0,9
OX40N,571274.0,10
XY25,438777.0,7
SR,258330.0,2
X5K,110099.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3803337.0,29
M-SC,1460294.0,17
H-LC,1273765.0,15
H-MC,1265766.0,15
M-LC,392666.0,4
M-MC,315466.0,2
L-SC,262048.0,3
L-MC,61160.0,1
L-LC,39639.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1243912.0      6
M-SC       XY24       834454.0      7
H-SC       AR         832766.0      5
           XR         813935.0      7
H-MC       XY24       583561.0      4
H-LC       X40        530501.0      5
H-SC       X40N       319562.0      4
M-MC       XY24       315466.0      2
H-SC       OX40N      262246.0      4
           SR         258330.0      2
H-LC       X40N       243964.0      3
H-MC       X40        218556.0      4
H-LC       AR         206980.0      2
H-MC       XY25       179836.0      2
L-SC       XR         173317.0      2
M-SC       AR         171922.0      2
M-LC       XY24       171376.0      2
M-SC       OX40N      147026.0      4
           XR         129608.0      2
H-LC       XY25       119268.0      3
M-LC       XR         111191.0      1
           X5K        110099.0      1
H-MC       XR         104420.0      1
M-SC       XY25       100034.0      1
H-LC       X200        94524.0      1
L-SC       OX40N       88731.0      1
H-MC       X40N        87575.0      2
H-LC       XY24        78528.0      1
M-SC       X40         77250.0      1
H-MC       OX40N       73271.0      1
H-SC       MH          72586.0      1
L-MC       XR          61160.0      1
L-LC       XY25        39639.0      1
H-MC       MH          18547.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
